In [11]:
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm

In [12]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [13]:
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (80,80))
    return img

In [14]:
TRAIN_PATH = 'train_img/'
TEST_PATH = 'test_img/'

In [15]:
train_img, test_img = [],[]
for img_path in tqdm(train['image_id'].values):
    train_img.append(read_img(TRAIN_PATH + img_path + '.png'))

for img_path in tqdm(test['image_id'].values):
    test_img.append(read_img(TEST_PATH + img_path + '.png'))

100%|██████████| 1732/1732 [00:26<00:00, 74.71it/s]


In [16]:
x_train = np.array(train_img, np.float32) / 255.
x_test = np.array(test_img, np.float32) / 255.

In [17]:
label_list = train['label'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(label_list))}
y_train = [Y_train[k] for k in label_list]   
y_train = np.array(y_train)

In [18]:
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [19]:
y_train = to_categorical(y_train)

In [20]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(80, 80, 3))

In [21]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='tanh'))
add_model.add(Dense(y_train.shape[1], activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 80, 80, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 80, 80, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 80, 80, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 40, 40, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 40, 40, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 40, 40, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 20, 20, 128)       0         
__________

In [22]:
batch_size = 16 # tune it
epochs = 22 # increase it

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)

In [24]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

Epoch 1/22
200/200 [==============================] - 282s - loss: 0.3930 - acc: 0.9041   

/usr/local/src/conda3_runtime.v18/4.1.1/lib/python3.5/site-packages/keras/callbacks.py:405: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)



Epoch 2/22
200/200 [==============================] - 282s - loss: 0.3371 - acc: 0.9259   
Epoch 3/22
200/200 [==============================] - 291s - loss: 0.2944 - acc: 0.9362   
Epoch 4/22
200/200 [==============================] - 281s - loss: 0.2836 - acc: 0.9412   
Epoch 5/22
200/200 [==============================] - 282s - loss: 0.2714 - acc: 0.9459   
Epoch 6/22
200/200 [==============================] - 285s - loss: 0.2315 - acc: 0.9562   
Epoch 7/22
200/200 [==============================] - 283s - loss: 0.2265 - acc: 0.9525   
Epoch 8/22
200/200 [==============================] - 285s - loss: 0.2007 - acc: 0.9681   
Epoch 9/22
200/200 [==============================] - 281s - loss: 0.1815 - acc: 0.9700   
Epoch 10/22
200/200 [==============================] - 283s - loss: 0.1703 - acc: 0.9731   
Epoch 11/22
200/200 [==============================] - 282s - loss: 0.1539 - acc: 0.9794   
Epoch 12/22
200/200 [==============================] - 282s - loss: 0.1413 - acc: 0.981

In [25]:
predictions = model.predict(x_test)

In [26]:
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in Y_train.items()}
pred_labels = [rev_y[k] for k in predictions]

In [27]:
vgg = pd.DataFrame({'image_id':test.image_id, 'label':pred_labels})
vgg.to_csv('sub_vgg.csv', index=False)

In [28]:
from IPython.display import HTML
import base64
def create_download_link( df, title = "Download CSV file", filename = "sub_vgg.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(vgg)  